In [1]:
# This is a package in preview.
from azureml.opendatasets import PublicHolidays

from datetime import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta
import numpy as np


end_date = datetime.strptime('2021-12-31', '%Y-%m-%d')
start_date = datetime.strptime('2020-01-01', '%Y-%m-%d')
hol = PublicHolidays(start_date=start_date, end_date=end_date)
hol_df = hol.to_pandas_dataframe()

[Info] read from /tmp/tmpnu_x1i3g/https%3A/%2Fazureopendatastorage.azurefd.net/holidaydatacontainer/Processed/part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


In [2]:
hol_df.head(120).tail()

,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
26053,Denmark,Skærtorsdag,Skærtorsdag,None,DK,2020-04-09
26054,Norway,Skjærtorsdag,Skjærtorsdag,None,NO,2020-04-09
26055,Argentina,Semana Santa (Viernes Santo) [Holy day (Holy ...,Semana Santa (Viernes Santo) [Holy day (Holy ...,None,AR,2020-04-10
26056,Australia,Good Friday,Good Friday,None,AU,2020-04-10
26057,Brazil,Sexta-feira Santa,Sexta-feira Santa,None,BR,2020-04-10


In [3]:
hol_df.to_csv('holidays.csv', index=False)

In [4]:
hol_df["Date"] = hol_df["date"]

In [5]:
# CountryName,RegionName,Date,HasHoliday

In [6]:
import pandas as pd

In [7]:
latest_df = pd.read_csv('lstm/data/OxCGRT_latest.csv',
                                parse_dates=['Date'],
                                encoding="ISO-8859-1",
                                dtype={"RegionName": str,
                                       "RegionCode": str},
                                error_bad_lines=False)

In [8]:
latest_df["GeoID"] = np.where(latest_df["RegionName"].isnull(),
                                      latest_df["CountryName"],
                                      latest_df["CountryName"] + ' / ' + latest_df["RegionName"])

In [9]:
all_countries = latest_df.CountryName.dropna().unique()
all_regions = {}
for country in all_countries:
    regions = latest_df[latest_df.CountryName == country].RegionName.dropna().unique()
    for region in regions:
        all_regions[region] = country

In [10]:
hol_df["CountryName"] = hol_df.countryOrRegion.astype(str).map(lambda x: all_regions[x] if x in all_regions.keys() else x)
hol_df["RegionName"] = hol_df.countryOrRegion.astype(str).map(lambda x: x if x in all_regions.keys() else np.nan)

In [35]:
hol_df.countryOrRegion.dropna().unique()

array(['Argentina', 'Australia', 'Austria', 'Belarus', 'Belgium',
       'Brazil', 'Canada', 'Colombia', 'Croatia', 'Czech', 'Denmark',
       'England', 'Finland', 'France', 'Germany', 'Hungary', 'Ireland',
       'Isle of Man', 'Italy', 'Japan', 'Mexico', 'Netherlands',
       'New Zealand', 'Northern Ireland', 'Norway', 'Poland', 'Portugal',
       'Scotland', 'Slovenia', 'South Africa', 'Spain', 'Sweden',
       'Switzerland', 'Ukraine', 'United Kingdom', 'United States',
       'Wales', 'India'], dtype=object)

In [11]:
# if country and nan has holiday set to true, else if country and region has holiday, set to true, otherwise false

In [12]:
# if country has holiday, region does too. If region has holiday, country does not

In [14]:
import math

In [41]:
def normalise_row(row):
    hol_df_regions = hol_df.countryOrRegion.dropna().unique()
    if (not row.CountryName in hol_df_regions and not row.RegionName in hol_df_regions):
        return np.NaN
    if ((hol_df.RegionName.map(lambda x: x == row.RegionName) & (row.Date == hol_df.Date) & (row.CountryName == hol_df.CountryName))).any():
        return 1.0
    if ((row.Date == hol_df.Date) & (row.CountryName == hol_df.CountryName) & (hol_df.RegionName.map(lambda x: np.isnan(x) if type(x) != str and type(row.RegionName) != str else False))).any():
        return 1.0
    return 0.0

In [42]:
%%time
# conditions = [((hol_df['CountryName'] == latest_df['CountryName']) & (hol_df['Date'] == latest_df['Date']) & (hol_df['RegionName'] == np.nan)).any(), ((hol_df['CountryName'] == latest_df['CountryName']) & (hol_df['Date'] == latest_df['Date']) & (hol_df['RegionName'] == latest_df['RegionName'])).any()]
# values = [True, True]
# latest_df["HasHoliday"] = np.select(conditions, values, default=False)
# latest_df["HasHoliday"] = np.where(((hol_df.CountryName == latest_df.CountryName) & (hol_df.Date.values == latest_df.Date.values) & (hol_df.RegionName == np.nan)).any(), True, False)
math.isnan(hol_df.loc[(hol_df.CountryName == "United States")].RegionName.values[0])
# (("United States" == hol_df.CountryName) & ("2020-07-04" == hol_df.Date) & (hol_df.RegionName.map(lambda x: np.isnan(x) if type(x) != str else x))).any()
latest_df['HasHoliday'] = latest_df.apply(lambda row : normalise_row(row), axis=1)

CPU times: user 9min 21s, sys: 588 ms, total: 9min 21s
Wall time: 9min 22s


In [44]:
latest_df.loc[latest_df.HasHoliday == True]

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,GeoID,test,HasHoliday
2118,Argentina,ARG,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Argentina,True,1.0
2172,Argentina,ARG,NaN,NaN,NAT_TOTAL,2020-02-24,0.0,NaN,0.0,NaN,...,14.29,6.67,6.67,7.69,7.69,0.0,0.0,Argentina,True,1.0
2173,Argentina,ARG,NaN,NaN,NAT_TOTAL,2020-02-25,0.0,NaN,0.0,NaN,...,14.29,6.67,6.67,7.69,7.69,0.0,0.0,Argentina,True,1.0
2201,Argentina,ARG,NaN,NaN,NAT_TOTAL,2020-03-24,3.0,1.0,3.0,1.0,...,100.00,72.22,72.22,79.49,79.49,25.0,25.0,Argentina,True,1.0
2210,Argentina,ARG,NaN,NaN,NAT_TOTAL,2020-04-02,3.0,1.0,3.0,1.0,...,100.00,78.89,78.89,79.49,79.49,75.0,75.0,Argentina,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97948,South Africa,ZAF,NaN,NaN,NAT_TOTAL,2020-06-16,2.0,1.0,2.0,1.0,...,88.10,74.44,74.44,74.36,74.36,75.0,75.0,South Africa,NaN,1.0
98002,South Africa,ZAF,NaN,NaN,NAT_TOTAL,2020-08-09,2.0,1.0,2.0,1.0,...,88.10,76.67,76.67,76.92,76.92,75.0,75.0,South Africa,NaN,1.0
98003,South Africa,ZAF,NaN,NaN,NAT_TOTAL,2020-08-10,2.0,1.0,2.0,1.0,...,88.10,76.67,76.67,76.92,76.92,75.0,75.0,South Africa,NaN,1.0
98048,South Africa,ZAF,NaN,NaN,NAT_TOTAL,2020-09-24,1.0,0.0,1.0,0.0,...,55.95,55.00,55.00,51.92,51.92,75.0,75.0,South Africa,NaN,1.0


In [43]:
latest_df[["CountryName", "RegionName", "Date", "HasHoliday"]].to_csv('holidays_computed.csv', index=False)

In [46]:
hol_pre_df = pd.read_csv('lstm/data/holidays_computed.csv')
last_date = hol_pre_df.Date.max()

In [50]:
df_done = latest_df.loc[latest_df['Date'] <= last_date]
df_to_do = latest_df.loc[latest_df['Date'] > last_date]
len(df_to_do)

0

In [49]:
df_to_do['HasHoliday'] = df_to_do.apply(lambda row: np.NaN if (not row.CountryName in hol_df_regions and not row.RegionName in hol_df_regions)
        else True if ((hol_df.RegionName.map(lambda x: x == row.RegionName) & (row.Date == hol_df.Date) & (row.CountryName == hol_df.CountryName))).any()
        else True if ((row.Date == hol_df.Date) & (row.CountryName == hol_df.CountryName) & (hol_df.RegionName.map(lambda x: np.isnan(x) if type(x) != str and type(row.RegionName) != str else False))).any()
        else False, axis=1)